# Imports

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime
import wmfdata
import pandas as pd
import requests
from wmfdata import hive,spark
from wmfdata.utils import print_err, pd_display_all
from wmfdata.utils import check_kerberos_auth, ensure_list


# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
# METRICS_MONTH_TEXT = '2019-07'
# MEDIAWIKI_HISTORY_SNAPSHOT = '2019-07'

last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)


METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")

MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")

# Preparation

In [3]:

# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = pd.Period(METRICS_MONTH_TEXT)
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    "metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str(metrics_month.asfreq("D", how="start")),
    "metrics_month_end": str((metrics_month + 1).start_time),
    "metrics_month_last_day": str(metrics_month.asfreq("D", how="end")),
    "api_metrics_month_first_day": metrics_month.asfreq("D", how="start").strftime("%Y%m%d"),
    "api_metrics_month_day_after": (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d"),
    "metrics_prev_month": str(metrics_month - 1),
    "retention_cohort": str(metrics_month - 2)
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

In [4]:
date_params

{'mediawiki_history_snapshot': '2021-09',
 'metrics_month': '2021-09',
 'metrics_month_start': '2021-09-01 00:00:00',
 'metrics_month_first_day': '2021-09-01',
 'metrics_month_end': '2021-10-01 00:00:00',
 'metrics_month_last_day': '2021-09-30',
 'api_metrics_month_first_day': '20210901',
 'api_metrics_month_day_after': '20211001',
 'metrics_prev_month': '2021-08',
 'retention_cohort': '2021-07'}

# MariaDB and Hive query metrics

In [5]:
queries = {
    "active_editors": {
        "file": "queries/active_editors.sql",
        "engine": "hive"
    },
    "edits": {
        "file": "queries/edits.hql",
        "engine": "hive"
    },
    "new_editor_retention": {
        "file": "queries/new_editor_retention.hql",
         "engine": "hive"
    },
    "global_south_edits_editors": {
        "file": "queries/global_south_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_edits_editors": {
        "file": "queries/mobile-heavy_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_new_editor_retention": {
        "file": "queries/mobile-heavy_new_editor_retention.hql",
        "engine": "hive"
    },
    "global_south_new_editor_retention": {
        "file": "queries/global_south_new_editor_retention.hql",
        "engine": "hive"
    }
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = spark.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running active_editors on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running edits on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running new_editor_retention on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_south_edits_editors on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running mobile-heavy_edits_editors on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running mobile-heavy_new_editor_retention on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_south_new_editor_retention on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


# Content metrics via API

In [6]:
NEW_PAGES_API = (
    # Replaces "https://wikimedia.org/api/rest_v1/metrics/" due to https://phabricator.wikimedia.org/P8605
    #"http://aqs1004.eqiad.wmnet:7232/analytics.wikimedia.org/v1/" 
    "https://wikimedia.org/api/rest_v1/metrics/"
    "edited-pages/new/{project}/all-editor-types/{page_type}/monthly/{start}/{end}"
)

headers = {
    "User-Agent": "https://github.com/wikimedia-research/Editing-movement-metrics (bot)"
}

# Create container for results
api_results = []

def get_new_pages(
    project="all-projects",
    page_type="content",
    start= date_params["api_metrics_month_first_day"],
    end= date_params["api_metrics_month_day_after"]
):
    url = NEW_PAGES_API.format(
        project = project,
        page_type = page_type,
        start = start,
        end = end
    )
    
    r = requests.get(url, headers=headers)
    data = r.json()["items"][0]["results"]
    frame = pd.DataFrame(data)
    frame["timestamp"] = pd.to_datetime(frame["timestamp"]).dt.tz_localize(None)
    frame = frame.rename(columns={"timestamp": "month"})
    
    return frame

In [7]:
project="all-projects",
page_type="content"
start=date_params["api_metrics_month_first_day"]
end=date_params["api_metrics_month_day_after"]

In [8]:
 url = NEW_PAGES_API.format(
        project = project,
        page_type = page_type,
        start = start,
        end = end
    )

## Total

In [9]:
total_new = get_new_pages().rename(columns={"new_pages": "net_new_content_pages"})
api_results.append(total_new)

## Wikidata

In [10]:
new_wd = get_new_pages(
    project="wikidata.org"
).rename(columns={
    "new_pages": "net_new_Wikidata_entities"
})
api_results.append(new_wd)

## Commons

In [11]:
new_commons = get_new_pages(
    project="commons.wikimedia.org"
).rename(columns={
    "new_pages": "net_new_Commons_content_pages"
})
api_results.append(new_commons)

## Wikipedias

In [12]:
# Get a list of project URLs (each one in a 1-tuple)
wp_domains = spark.run("""
select domain_name
from canonical_data.wikis
where database_group = "wikipedia"
""", format="raw")

# Query the API for each project and append records to a list
results = []
n = len(wp_domains)

/usr/lib/anaconda-wmf/lib/python3.7/site-packages/wmfdata/spark.py:304: FutureWarning: The 'raw' format is deprecated. It will be removed in the next major release.
  category=FutureWarning
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [13]:

for idx, val in enumerate(wp_domains):
    domain = val[0]
    
    if idx % 10 == 0:
        msg = "Now on the {}th project of {} ({})"
        print_err(msg.format(idx, n, domain))
        
    frame = get_new_pages(project=domain).reset_index()
    frame["project"] = domain
    records = frame.to_dict("records")
    results.extend(records)
    
    # Sleep 20 milliseconds
    time.sleep(0.02)

# Turn the big list of records into a data frame
new_per_wp = pd.DataFrame(results)

# Sum across projects to get new Wikipedia articles per month
new_wp = new_per_wp.groupby("month").agg(
    {"new_pages": "sum"}
).rename(columns={"new_pages": "net_new_Wikipedia_articles"}).reset_index()

api_results.append(new_wp)

Now on the 0th project of 312 (aa.wikipedia.org)
Now on the 10th project of 312 (arc.wikipedia.org)
Now on the 20th project of 312 (ay.wikipedia.org)
Now on the 30th project of 312 (bg.wikipedia.org)
Now on the 40th project of 312 (bug.wikipedia.org)
Now on the 50th project of 312 (chy.wikipedia.org)
Now on the 60th project of 312 (da.wikipedia.org)
Now on the 70th project of 312 (en.wikipedia.org)
Now on the 80th project of 312 (fj.wikipedia.org)
Now on the 90th project of 312 (gcr.wikipedia.org)
Now on the 100th project of 312 (hak.wikipedia.org)
Now on the 110th project of 312 (hu.wikipedia.org)
Now on the 120th project of 312 (ilo.wikipedia.org)
Now on the 130th project of 312 (kaa.wikipedia.org)
Now on the 140th project of 312 (km.wikipedia.org)
Now on the 150th project of 312 (kw.wikipedia.org)
Now on the 160th project of 312 (li.wikipedia.org)
Now on the 170th project of 312 (map-bms.wikipedia.org)
Now on the 180th project of 312 (mnw.wikipedia.org)
Now on the 190th project of 3

In [14]:
# Strip timezones returned by API so our month columns merge nicely
for df in api_results:
    df["month"] = df["month"].dt.tz_localize(None)

# Combining and saving metrics

In [15]:
queries["edits"]["result"] = queries["edits"]["result"].rename({"data_edits": "wikidata_edits"}, axis=1)

In [16]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]
results.extend(api_results)

In [17]:
# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail())

,active_editors,anonymous_edits,global_south_active_editors,global_south_edits,global_south_new_editor_retention,global_south_nonbot_edits,mobile-heavy_wiki_active_editors,mobile-heavy_wiki_edits,mobile-heavy_wiki_new_editor_retention,mobile-heavy_wiki_nonbot_edits,mobile_edits,net_new_Commons_content_pages,net_new_Wikidata_entities,net_new_Wikipedia_articles,net_new_content_pages,new_active_editors,new_editor_retention,non_anonymous_edits,other_nonbot_edits,returning_active_editors,revert_rate,total_edits,uploads,wikidata_edits
month,,,,,,,,,,,,,,,,,,,,,,,,
2021-05-01,98730.0,2294554.0,25308.0,3414303.0,0.05619,2700943.0,5560.0,1226707.0,NaN,641991.0,2050899.0,1218483.0,321578.0,241237.0,2227436.0,19876.0,0.073751,47328592.0,5280862.0,78854.0,0.053275,49624611.0,1270095.0,20602931.0
2021-06-01,91670.0,2089434.0,24610.0,3122716.0,0.0571646,2530074.0,5103.0,963573.0,0.061350,567731.0,1899060.0,750711.0,349563.0,325849.0,1739466.0,17723.0,0.066024,40073089.0,5218602.0,73947.0,0.055657,42163576.0,796256.0,18875684.0
2021-07-01,89307.0,2049005.0,25186.0,3184875.0,0.0539202,2744544.0,5320.0,1144866.0,0.060699,598855.0,2006736.0,694958.0,341418.0,260784.0,2252121.0,16397.0,0.059166,42730252.0,5250912.0,72910.0,0.053741,44780394.0,738459.0,18720677.0
2021-08-01,90312.0,2100177.0,25249.0,3594274.0,NaN,2735392.0,5606.0,1322700.0,0.058824,618020.0,2083814.0,1258907.0,582256.0,247061.0,2331908.0,16299.0,0.060366,45811721.0,5587077.0,74013.0,0.050318,47913055.0,1311357.0,19905473.0
2021-09-01,88925.0,2052391.0,24011.0,3010823.0,NaN,2498736.0,5164.0,997518.0,0.055482,586738.0,1946403.0,808814.0,358116.0,239849.0,1684130.0,17042.0,0.062776,37448564.0,6045787.0,71883.0,0.054839,39502236.0,899197.0,15241328.0


In [18]:
metrics.to_csv(FILENAME, sep="\t")